## Save and load the model

In [8]:
import tensorflow as tf

from aMNWtModel import AMNWtModel, RNNCell
model_class = AMNWtModel


# from aMRNNModel import AMRNNModel, RNNCell
# model_class = AMRNNModel
# model_file = "Models/AMRNN_model.keras"


seed = 10
tf.random.set_seed(seed)


# dataset_file = "./Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']
# epoch = 20
# batch_size = 30
# uptake_max_index = None
# model_file = "Models/AMNWt_IJN1463_10_UB.keras"
# model_file = "Models/AMNWt_IJN1463_10_UB_no_scaling.keras"


dataset_file = "./Dataset/e_coli_core_UB.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']
epoch = 20 #200
batch_size = 7
uptake_max_index = None
model_file = "Models/AMNWt_e_coli_core_UB.keras"
model_file = "Models/AMNWt_e_coli_core_UB_no_scaling.keras"


# dataset_file = "./Dataset/biolog_iML1515_EXP_UB.npz"
# objective=['BIOMASS_Ec_iML1515_core_75p37M']
# epoch = 1 #20
# batch_size = 30
# uptake_max_index=151

# Dataset plus model structure
print("---------------------------------------- model ----------------------------------------")
model = model_class(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   verbose=True,
                )
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()


# Construct and train an AMNWt model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)


2023-10-19 10:44:32.559362: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 10:44:32.562373: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-10-19 10:44:32.788483: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


R2 : 0.9668980416160837
Q2 : 0.9644709223002214


In [9]:
from tools import custom_loss

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file, compile and train it. First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
my_mse = custom_loss(model.S, model.P_out,model.P_in)
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9668980416160837
Q2 : 0.9644709223002214


In [10]:
# Save the train AMNWt_model
model_file = "Models/AMNWt_e_coli_core_UB_to_delete.keras"
tf.keras.models.save_model(AMNWt_model,model_file, overwrite=True, save_format=None, save_traces=True)

In [11]:
from tools import custom_loss

AMNWt_model_ = tf.keras.models.load_model(model_file, 
                                          custom_objects={"RNNCell":RNNCell,
                                                          "my_mse":custom_loss(model.S, 
                                                                               model.P_out,
                                                                               model.P_in)}
                                          )

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9668980416160837
Q2 : 0.9644709223002214
R2 : 0.9839517517947176
Q2 : 0.9807428905171167
